<a href="https://colab.research.google.com/github/alistairwalsh/2015-05-04-swinpython/blob/add-config-wiki-link/Copy_of_auto_sklearn_AMIR19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto-Sklearn

auto-sklearn is an automated machine learning toolkit and a drop-in replacement for a scikit-learn estimator.
It frees a machine learning user from algorithm selection and hyperparameter tuning. It leverages recent advantages in Bayesian optimization, meta-learning and ensemble construction.

[Docu](https://automl.github.io/auto-sklearn/master/),
[Paper](https://papers.nips.cc/paper/5872-efficient-and-robust-automated-machine-learning)

Notebook author: Marius Lindauer ([www.automl.org](https://www.automl.org))

# Installation

The notebook was created based on 
scikit-learn 0.19.2, smac 0.8.0 and auto-sklearn 0.5.1.

In [ ]:
!apt-get install swig -y
!pip install Cython numpy

# sometimes you have to run the next command twice on colab
# I haven't figured out why
!pip install auto-sklearn

In [ ]:
# ignore some annoying warnings for demonstrating auto-sklearn 
# shouldn't be done in real production
import numpy as np
np.warnings.filterwarnings('ignore')

# 1st Example

We want to train a classifier for the [digits](http://archive.ics.uci.edu/ml/datasets/Pen-Based+Recognition+of+Handwritten+Digits) dataset.
All we do is to split the dataset into training and test data,
pass the training data to auto-sklearn
and evaluate the trained model on test data.


In [1]:
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

# Load data
X, y = sklearn.datasets.load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(X, y, random_state=1)

# What our data looks like

![Imgur](https://i.imgur.com/6zXnekR.png)

![Imgur](https://i.imgur.com/Tq8MIq9.png)

# What it really is

![Imgur](https://i.imgur.com/zJHxoTy.png)

![Imgur](https://i.imgur.com/CDsV5Fu.jpg)

In [ ]:
import autosklearn.classification

# configure auto-sklearn
automl = autosklearn.classification.AutoSklearnClassifier(
          time_left_for_this_task=120, # run auto-sklearn for at most 2min
          per_run_time_limit=30, # spend at most 30 sec for each model training
          )

# train model(s)
automl.fit(X_train, y_train)

# evaluate
y_hat = automl.predict(X_test)
test_acc = sklearn.metrics.accuracy_score(y_test, y_hat)
print("Test Accuracy score {0}".format(test_acc))

The accuracy might not be quite state-of-the-art after running auto-sklearn only for two minutes. If you want to achieve better results, please try to increase the time limit `time_left_for_this_task`.

## Inspect some statistics of our first example

Please note that auto-sklearn has internally used a holdout set of the traning set to estimate the quality of the trained model. Based on this hold-out validation set, auto-sklearn reports a validation score.

In [ ]:
print(automl.sprint_statistics())

## Inspect Ensemble

Auto-sklearn considers all trained models as potential candidates to build an ensemble out of these.
The following command allows you to see the ensemble.

Since the ensemble does not use a simple majority voting, but a weighted ensemble,
the fomat is the following:

`(ensemble weight, machine learning pipeline)`

In [ ]:
print(automl.show_models())

# 2nd Example: Holdout resampling

Now, let's switch to a different dataset, breast_cancer

In [ ]:
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

import autosklearn.classification

X, y = sklearn.datasets.load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = \
    sklearn.model_selection.train_test_split(X, y, random_state=1)

automl = autosklearn.classification.AutoSklearnClassifier(
          time_left_for_this_task=120,
          per_run_time_limit=30,
          disable_evaluator_output=False,
          resampling_strategy='holdout',
          resampling_strategy_arguments={'train_size': 0.80}
          )

automl.fit(X_train, y_train, dataset_name='breast_cancer')

y_hat = automl.predict(X_test)
test_acc = sklearn.metrics.accuracy_score(y_test, y_hat)
print("Test Accuracy score: {0}".format(test_acc))